In [19]:
import json

from data_prepare import samples
from datasets import load_dataset

from transformers import AutoTokenizer, AutoModelForCausalLM
from transformers import BitsAndBytesConfig
from transformers import Trainer, TrainingArguments
from transformers import pipeline

from peft import get_peft_model, LoraConfig, TaskType, PeftModel


# Model

In [40]:
model_name = '/Users/kaichen/workspace/data/models/deepseekr1-1.5b'

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)
# model = AutoModelForCausalLM.from_pretrained(model_name).to('cuda')


# Data

In [41]:
with open('datasets.jsonl', 'w', encoding='utf-8') as f:
    for sample in samples:
        json_line = json.dumps(sample, ensure_ascii=False)
        f.write(json_line + '\n')
    else:
        print('Data saved to datasets.jsonl')   


Data saved to datasets.jsonl


# Train and test set

In [42]:
dataset = load_dataset("json", data_files={"train": "datasets.jsonl"}, split="train")

Generating train split: 50 examples [00:00, 1283.07 examples/s]


In [43]:
len(dataset)

50

In [44]:
train_test_split = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = train_test_split['train']
test_dataset = train_test_split['test']


In [45]:
print(len(train_dataset))
print(len(test_dataset))

45
5


# Tokenizer




该函数的作用是将数据集中的样本转换为模型训练所需的格式，主要包括文本合并、分词处理及标签生成。以下是详细解释：

### 函数功能分解
1. **文本合并**：
   ```python
   text = [f"{prompt}\n{completion}" for prompt, completion in zip(examples['prompt'], examples['completion'])]
   ```
   - 将每个样本的 `prompt`（输入提示）和 `completion`（预期输出）通过换行符 `\n` 拼接成一个完整文本。
   - 例如：若 `prompt` 为“写一首诗”，`completion` 为“春眠不觉晓”，合并后为“写一首诗\n春眠不觉晓”。

2. **分词处理**：
   ```python
   tokens = tokenizer(text, padding="max_length", truncation=True, max_length=512)
   ```
   - 使用预训练的 `tokenizer` 对合并后的文本进行分词。
   - **填充 (Padding)**：将所有序列填充到固定长度（`max_length=512`），确保批量输入时形状统一。
   - **截断 (Truncation)**：若文本超过 512 个 Token，自动截断以适配模型最大长度限制。

3. **生成标签**：
   ```python
   tokens['labels'] = tokens['input_ids'].copy()
   ```
   - 将分词后的 `input_ids`（Token ID 序列）直接复制为 `labels`，用于监督学习。
   - 在自回归语言模型（如 GPT）中，模型的目标是根据上文预测下一个 Token，此时标签应与输入序列一致，但实际训练时需在模型内部将标签右移一位。

### 为什么需要这个函数？
1. **数据格式化**：
   - 模型无法直接处理原始文本，需转换为 Token ID 序列。此函数统一了数据格式，确保所有样本符合模型输入要求。

2. **长度控制**：
   - 通过填充和截断，处理不同长度的文本，避免训练时因序列长度不一致导致的错误。

3. **标签生成**：
   - 为监督学习提供目标标签。在生成任务中，标签帮助模型学习如何从 `prompt` 生成正确的 `completion`。

### 潜在问题与改进
- **标签遮盖**：
  - **问题**：当前函数未区分 `prompt` 和 `completion` 部分的标签，导致模型可能学习预测 `prompt` 本身，而非仅生成 `completion`。
  - **改进**：若需模型仅关注 `completion` 的生成，应将 `prompt` 部分的标签设为 `-100`（损失计算时忽略）。例如：
    ```python
    prompt_tokens = tokenizer(examples['prompt'], add_special_tokens=False)
    prompt_length = len(prompt_tokens['input_ids']) + 1  # +1 为换行符
    labels = [
        [-100] * prompt_length + input_ids[prompt_length:] 
        for input_ids in tokens['input_ids']
    ]
    tokens['labels'] = labels
    ```

### 适用场景
- **预训练任务**：模型需学习完整文本的分布（包括 `prompt` 和 `completion`）。
- **生成任务（需调整）**：若需模型根据 `prompt` 生成 `completion`，需修改标签以屏蔽 `prompt` 部分的损失计算。

### 总结
该函数是数据预处理的核心步骤，负责将原始文本转换为模型可训练的格式，但在实际任务中可能需要根据训练目标调整标签生成逻辑。


In [46]:
def tokenizer_function(examples, tokenizer):
    text = [f"{prompt}\n{completion}" for prompt, completion in zip(examples['prompt'], examples['completion'])]
    tokens = tokenizer(text, padding="max_length", truncation=True, max_length=512)
    tokens['labels'] = tokens['input_ids'].copy()

    return tokens


In [47]:
tokenized_train_dataset = train_dataset.map(lambda examples: tokenizer_function(examples, tokenizer), batched=True)
tokenized_eval_dataset = test_dataset.map(lambda examples: tokenizer_function(examples, tokenizer), batched=True)



Map: 100%|██████████| 5/5 [00:00<00:00, 1545.09 examples/s]


In [48]:
print(tokenized_train_dataset[0])

{'prompt': 'Question 28: Why is it important to practice dynamics in singing?', 'completion': 'Answer 28: Dynamics add emotion and variety to your performance, making your singing more expressive and engaging.', 'input_ids': [151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643

# Quantization

GPU required

### `BitsAndBytesConfig` 的详细解释

#### **作用**
`BitsAndBytesConfig` 是 Hugging Face Transformers 库中用于配置 **模型量化（Quantization）** 的类，其核心目标是通过降低模型参数的精度（如将 32 位浮点数转换为 8 位或 4 位整数），显著减少模型的内存占用，使得大型语言模型（如 LLaMA、GPT）能够在资源受限的设备（如消费级 GPU 或 CPU）上运行。

#### **核心功能**
1. **内存优化**：
   - 将模型权重从 `float32` 转换为 `int8` 或 `int4`，内存占用减少至原来的 1/4 或 1/8。
   - 例如：一个 10GB 的模型，8 位量化后仅需约 2.5GB，4 位量化仅需约 1.25GB。

2. **推理加速**：
   - 量化后的模型在支持低精度计算的硬件（如 NVIDIA GPU）上可能更快，但需权衡精度损失。

3. **灵活配置**：
   - 支持混合精度（部分层保留高精度）、阈值控制（跳过小数值的量化）等高级设置。

---

### **参数详解**
以下是 `BitsAndBytesConfig` 的关键参数（以 8 位和 4 位量化为重点）：

| 参数 | 类型 | 作用 | 示例值 |
|------|------|------|--------|
| `load_in_8bit` | `bool` | 启用 8 位量化 | `True` |
| `load_in_4bit` | `bool` | 启用 4 位量化 | `True` |
| `llm_int8_threshold` | `float` | 阈值：超过此值的参数保留为浮点 | `6.0` |
| `bnb_4bit_quant_type` | `str` | 4 位量化的类型（`"fp4"` 或 `"nf4"`） | `"nf4"` |
| `bnb_4bit_compute_dtype` | `torch.dtype` | 4 位量化的计算数据类型 | `torch.float16` |
| `bnb_4bit_use_double_quant` | `bool` | 是否启用双重量化（进一步压缩） | `True` |

---

### **完整用法示例**

#### **1. 8 位量化**
```python
from transformers import AutoModelForCausalLM, BitsAndBytesConfig

# 配置 8 位量化，并设置量化阈值
quantization_config = BitsAndBytesConfig(
    load_in_8bit=True,
    llm_int8_threshold=6.0  # 绝对值超过 6.0 的参数保留为浮点
)

# 加载量化后的模型
model = AutoModelForCausalLM.from_pretrained(
    "meta-llama/Llama-2-7b-chat-hf",
    quantization_config=quantization_config,
    device_map="auto"  # 自动分配模型层到可用设备（GPU/CPU）
)
```

#### **2. 4 位量化（更高效）**
```python
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",          # 使用 4 位 NormalFloat 量化
    bnb_4bit_compute_dtype=torch.float16,  # 计算时使用 float16 加速
    bnb_4bit_use_double_quant=True      # 启用双重量化（二次压缩）
)

model = AutoModelForCausalLM.from_pretrained(
    "mistralai/Mistral-7B-v0.1",
    quantization_config=quantization_config,
    device_map="auto"
)
```

---

### **使用场景**
| 场景 | 推荐配置 | 说明 |
|------|----------|------|
| 低显存 GPU 推理 | `load_in_8bit=True` | 平衡内存和精度 |
| 超大规模模型部署 | `load_in_4bit=True` + 双重量化 | 极致压缩内存 |
| 高精度需求场景 | `llm_int8_threshold=6.0` | 保留重要参数为浮点 |

---

### **注意事项**
1. **依赖安装**：
   ```bash
   pip install bitsandbytes  # 必须安装量化支持库
   pip install accelerate     # 用于设备自动分配（device_map="auto"）
   ```

2. **性能权衡**：
   - 量化可能导致模型精度下降，尤其在生成任务中可能影响文本连贯性。
   - 4 位量化比 8 位更激进，内存更小，但精度损失更大。

3. **硬件兼容性**：
   - 量化模型在 NVIDIA GPU 上支持最佳，部分操作在 CPU 上可能无法加速。

---

### **完整总结**
`BitsAndBytesConfig` 是 Transformers 库中实现模型量化的核心配置类，通过将模型权重从高精度浮点数转换为低精度整数（8 位或 4 位），显著降低内存占用，使大模型能够在资源受限的环境中运行。其关键功能包括：

1. **内存压缩**：8 位量化减少内存至 1/4，4 位量化至 1/8。
2. **灵活配置**：支持阈值控制、混合精度、双重量化等高级选项。
3. **易用性**：只需在加载模型时传入配置，无需修改模型代码。

**示例总结**：
- **8 位量化**：适合大多数场景，平衡内存和精度。
- **4 位量化**：适合极致内存优化，需搭配 `nf4` 类型和 `float16` 计算以维持性能。

通过合理配置 `BitsAndBytesConfig`，开发者可以在消费级硬件上高效部署百亿参数级别的语言模型。

In [ ]:
quantization_config = BitsAndBytesConfig(load_in_8bit=True)

# device_map={"cuda:0": "cpu"}
# device_map={"auto"}
model = AutoModelForCausalLM.from_pretrained(model_name, config=quantization_config)
# model.save_pretrained(model_name)




# Lora 

GPU required

---

### **PEFT 库与 LoRA 方法详解**

#### **PEFT 库简介**
**PEFT**（Parameter-Efficient Fine-Tuning）是 Hugging Face 推出的库，旨在通过高效调整少量参数来微调大型预训练模型（如 GPT、LLaMA）。其核心目标是 **降低训练成本**（显存、算力），同时保持模型性能。常见应用场景包括：
- **资源受限环境**：在消费级 GPU（如 24GB 显存）上微调 10B+ 参数的模型。
- **多任务适配**：为不同任务生成轻量级适配器，无需存储完整模型副本。
- **避免灾难性遗忘**：仅修改部分参数，保留预训练模型的基础能力。

支持的高效微调方法包括：
- **LoRA**（Low-Rank Adaptation，低秩适应）
- **Adapter**（插入小型神经网络模块）
- **Prompt Tuning**（学习软提示向量）
- **IA3**（通过向量缩放激活值）

---

### **LoRA 方法原理**
LoRA 的核心思想是 **冻结原始模型权重**，仅通过低秩矩阵（Low-Rank Matrices）对权重更新进行参数化。具体来说：
1. 对于原始权重矩阵 \( W \in \mathbb{R}^{d \times k} \)，定义两个低秩矩阵 \( A \in \mathbb{R}^{d \times r} \) 和 \( B \in \mathbb{R}^{r \times k} \)，其中 \( r \ll \min(d,k) \)。
2. 权重更新量表示为 \( \Delta W = A \cdot B \)，最终输出为 \( W + \Delta W \)。
3. 训练时仅更新 \( A \) 和 \( B \)，原始 \( W \) 保持冻结。

![LoRA 示意图](https://miro.medium.com/v2/resize:fit:720/format:webp/1*_0fYSlZRlC8mAJi4Z3k1GQ.png)

---

### **`LoraConfig` 参数详解**
以下是 `LoraConfig` 的所有参数及其作用：

| 参数 | 类型 | 作用 | 影响 | 示例值 |
|------|------|------|------|--------|
| **`r`** | `int` | 低秩矩阵的秩（Rank） | 秩越大，可训练参数越多，灵活性增加，但可能过拟合 | `8`, `16`, `32` |
| **`lora_alpha`** | `int` | 低秩适应的缩放因子 | 控制低秩矩阵的权重强度，通常与 `r` 成比例 | `16`, `32`, `64` |
| **`lora_dropout`** | `float` | LoRA 层的 Dropout 率 | 正则化强度，防止过拟合 | `0.1`, `0.2` |
| **`task_type`** | `TaskType` | 任务类型 | 决定模型结构调整方式 | `CAUSAL_LM`, `SEQ_CLS` |
| **`target_modules`** | `List[str]` | 应用 LoRA 的模块列表 | 指定对哪些层进行低秩适应 | `["q_proj", "v_proj"]` |
| **`bias`** | `str` | 是否训练偏置项 | 控制是否更新偏置参数 | `"none"`, `"all"`, `"lora_only"` |
| **`modules_to_save`** | `List[str]` | 额外训练的模块（非 LoRA 层） | 允许特定模块完全参与训练 | `["lm_head"]` |
| **`layers_to_transform`** | `List[int]` | 应用 LoRA 的层索引 | 仅对指定层进行低秩适应 | `[0, 2, 4]` |
| **`layers_pattern`** | `str` | 层名称匹配模式 | 通过正则表达式选择目标层 | `".*decoder.*"` |

---

#### **关键参数解释**
1. **`r`（Rank）**:
   - **作用**：低秩矩阵的维度，直接影响可训练参数数量。
   - **示例**：若 `r=8`，每个 LoRA 层的参数量为 \( d \times 8 + 8 \times k \)。
   - **建议**：从较小的值（如 `8`）开始，逐步增加直到性能稳定。

2. **`lora_alpha`**:
   - **作用**：缩放低秩矩阵的权重，最终更新量为 \( \Delta W = \frac{\alpha}{r} \cdot A \cdot B \)。
   - **示例**：若 `alpha=32`, `r=16`，则缩放因子为 `2`。
   - **建议**：通常设置为 `r` 的 2~4 倍，如 `r=8` → `alpha=16`。

3. **`target_modules`**:
   - **作用**：指定模型中对哪些线性层（如注意力模块的 `q_proj`, `v_proj`）应用 LoRA。
   - **示例**：对 LLaMA 模型，通常选择 `["q_proj", "v_proj"]`。
   - **建议**：参考模型架构文档，选择与任务相关的模块。

4. **`modules_to_save`**:
   - **作用**：允许某些模块（如分类头 `lm_head`）在微调时完全训练（非 LoRA 模式）。
   - **示例**：在生成任务中，通常需要微调输出层：`modules_to_save=["lm_head"]`。

---

### **完整代码示例**
```python
from peft import LoraConfig, TaskType, get_peft_model
from transformers import AutoModelForCausalLM

# 初始化模型
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf")

# 配置 LoRA
lora_config = LoraConfig(
    r=16,                          # 低秩矩阵的秩
    lora_alpha=32,                 # 缩放因子 (alpha/r 决定实际缩放比例)
    lora_dropout=0.1,              # Dropout 率
    task_type=TaskType.CAUSAL_LM,   # 任务类型（因果语言模型）
    target_modules=["q_proj", "v_proj"],  # 目标模块（查询和值投影层）
    bias="none",                   # 不训练偏置项
    modules_to_save=["lm_head"],   # 额外训练输出层
)

# 应用 LoRA 到模型
model = get_peft_model(model, lora_config)

# 打印可训练参数占比
model.print_trainable_parameters()
# 输出示例: trainable params: 8,847,360 || all params: 6,775,656,448 || 0.13%
```

---

### **通俗总结**
1. **PEFT 库**：让你用“小修小补”的方式微调大模型，无需动全部参数，省钱省力。
2. **LoRA 方法**：通过给模型“打补丁”（低秩矩阵），只训练这些补丁来适应新任务。
   - **`r`**：补丁的大小，越大越灵活，但也可能“补过头”。
   - **`alpha`**：补丁的强度，和 `r` 配合使用（通常 `alpha=2*r`）。
   - **`target_modules`**：决定补丁打在哪里（如注意力层的特定位置）。
3. **参数选择**：
   - 从 `r=8`、`alpha=16` 开始，逐步调整。
   - 根据任务类型（如文本生成、分类）设置 `task_type` 和 `target_modules`。
   - 如果任务需要更新输出层（如分类头），使用 `modules_to_save`。

通过合理配置 `LoraConfig`，你可以在 24GB 显存的 GPU 上微调 70B 参数的模型！🚀

下面我们用通俗易懂的语言详细介绍一下 LoRA 及其各个参数的作用和效果，并在最后给出完整的总结。

---

## 一、LoRA 简介

**LoRA（Low-Rank Adaptation）** 是一种参数高效的微调方法。传统的微调方法往往需要更新整个大模型的参数，而 LoRA 的思路是保持原始预训练模型参数不变，仅通过增加一些额外的、较小的“低秩矩阵”（即参数量非常少的矩阵）来适配特定任务。这样做有两个好处：

- **节省计算资源和内存**：只训练少量新增参数，降低了显存和计算需求。  
- **提高微调效率**：在大模型上进行微调时，只更新新增的参数，训练速度更快，资源占用更少。

---

## 二、LoRA 的核心思想

在神经网络中，很多操作（比如全连接层、注意力层）都涉及大矩阵的乘法运算。LoRA 的主要思路是将这些大矩阵的更新表示为两个小矩阵的乘积。  
- 原始模型参数保持不变；  
- 增加的低秩矩阵经过训练来捕捉任务特定的信息；  
- 最终模型的输出由原始固定参数和低秩更新共同决定。

---

## 三、LoraConfig 中各参数的详细介绍

在使用 PEFT 库进行 LoRA 微调时，需要通过 `LoraConfig` 来指定相关参数。下面是主要参数的详细解释：

### 1. r
- **含义**：  
  `r` 表示低秩分解时低秩矩阵的秩（即低维空间的维度）。
  
- **效果与影响**：  
  - **较大 r**：意味着低秩矩阵的维度较高，可以捕捉更多细节信息，有助于提高微调后模型的表达能力；但同时也会增加新增参数的数量和计算量。  
  - **较小 r**：参数更少，计算和存储开销小，但可能在面对复杂任务时适应能力不足。

### 2. lora_alpha
- **含义**：  
  `lora_alpha` 是一个缩放因子，用来调节低秩矩阵更新在整体模型中的影响力。
  
- **效果与影响**：  
  - **作用**：在实际计算中，LoRA 模块生成的更新项会先乘以 `lora_alpha` 再与原始输出相加。  
  - **影响**：  
    - 较大的 `lora_alpha` 会放大低秩更新的效果，使得微调更新更为明显；  
    - 通常，实际影响体现在 `lora_alpha / r` 的比例上，这个比例决定了低秩更新和原始参数之间的平衡。

### 3. lora_dropout
- **含义**：  
  `lora_dropout` 指定了在训练过程中对低秩更新应用 dropout 的概率。
  
- **效果与影响**：  
  - **防止过拟合**：通过在训练时随机“丢弃”一部分低秩更新，可以增强模型的鲁棒性，避免对训练数据过拟合。  
  - **平衡更新**：合理设置 dropout 率有助于使得低秩更新更加稳定，但如果过高可能会使得适配效果减弱。

### 4. task_type
- **含义**：  
  `task_type` 指明了当前微调任务的类型，如因果语言建模（CAUSAL_LM）、序列分类（SEQ_CLS）、问答任务等。
  
- **效果与影响**：  
  - **模块选择**：不同的任务可能需要在模型中注入 LoRA 模块的位置不同。指定任务类型后，PEFT 库会自动选择合适的模块进行适配。  
  - **输入输出格式**：任务类型还能帮助确定训练时的输入输出、损失函数等设置，使得微调过程更为合理。

### 5. 其他可能参数（根据不同版本可能存在）
- **target_modules**：  
  - **含义**：明确指定哪些模型层（例如注意力层、前馈层）需要使用 LoRA 适配。  
  - **效果**：只在部分模块上应用 LoRA 可以进一步降低参数量，并专注于最关键的部分。
  
- **bias**：  
  - **含义**：决定是否对模型中的偏置项（bias）也采用 LoRA 进行适配。  
  - **效果**：控制是否更新偏置，有时更新偏置可能对任务效果有微小影响，根据任务需求决定是否更新。

- **inference_mode**（如果存在）：  
  - **含义**：用于指示是否在推理时也启用 LoRA 的特殊处理模式，以便优化推理速度。  
  - **效果**：在推理过程中可以禁用某些额外计算，从而加速模型预测。

---

## 四、完整通俗总结

LoRA 的核心思想是**不改变原始大模型的参数**，而是在关键层上添加两个小矩阵来“适配”任务。这样做的好处是：
- **高效微调**：只需要训练新增的低秩矩阵，大大减少了参数数量和计算资源需求。  
- **节省资源**：在显存有限或者计算能力受限的场景下，可以轻松微调大模型。  
- **灵活应用**：通过调整参数（如 `r`、`lora_alpha` 和 `lora_dropout`），可以平衡模型的适应能力和训练资源消耗。  
- **任务适配**：`task_type` 参数帮助自动选择需要调整的模型模块，使得 LoRA 能够在不同任务（如文本生成、分类、问答等）中发挥作用。

每个参数都有其具体的作用：
- **r** 决定了新增矩阵的大小，直接影响模型能捕捉的信息量和训练时的资源占用；
- **lora_alpha** 控制低秩更新的强度，帮助平衡更新效果；
- **lora_dropout** 通过随机丢弃部分更新，防止过拟合并增强训练稳定性；
- **task_type** 则告诉系统该在模型的哪些部分插入低秩适配器，确保适配方式与任务匹配。

总之，LoRA 为大模型的微调提供了一种既高效又经济的解决方案，通过仅调整一小部分参数，使得大模型在各种下游任务上也能快速适应，并且大大降低了训练资源和存储的要求。

In [32]:
lora_config = LoraConfig(
    r=16,  # Number of bits for the mantissa
    lora_alpha=32,  # Scaling factor for low-rank adaptation
    lora_dropout=0.1,  # Dropout rate for low-rank adaptation
    task_type=TaskType.CAUSAL_LM,  # Task type
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
# model.save_pretrained(model_name)

trainable params: 2,179,072 || all params: 1,779,267,072 || trainable%: 0.1225


# Train parameters setting

In [56]:
""" 
下面对每个参数进行简明通俗的解释：

- **output_dir**：指定训练过程中保存微调模型、检查点（checkpoint）等输出文件的目录。  
- **num_train_epochs**：整个训练过程中遍历训练数据集的总轮数。  
- **per_device_train_batch_size**：每个设备（例如每个 GPU）上一次训练使用的样本数。如果有多个设备，实际的批量大小会乘以设备数量。  
- **gradient_accumulation_steps**：每累计多少步梯度后再进行一次参数更新。这相当于增大了批量大小，有助于在显存较小的设备上训练大模型。  
- **fp16**：是否启用16位（混合精度）训练，相比默认的32位训练能加速训练并节省显存。  
- **eval_steps**：每隔多少训练步数进行一次验证评估，以便观察模型在验证集上的表现。  
- **learning_rate**：训练时使用的学习率，决定模型参数更新的幅度。  
- **logging_dir**：存储训练过程中的日志信息的目录，可以用来追踪和分析训练过程。  
- **logging_steps**：每隔多少步记录一次训练日志，帮助监控训练进度。  
- **run_name**：这次训练运行的名称，方便在日志和管理界面中区分不同的实验。
"""

' \n下面对每个参数进行简明通俗的解释：\n\n- **output_dir**：指定训练过程中保存微调模型、检查点（checkpoint）等输出文件的目录。  \n- **num_train_epochs**：整个训练过程中遍历训练数据集的总轮数。  \n- **per_device_train_batch_size**：每个设备（例如每个 GPU）上一次训练使用的样本数。如果有多个设备，实际的批量大小会乘以设备数量。  \n- **gradient_accumulation_steps**：每累计多少步梯度后再进行一次参数更新。这相当于增大了批量大小，有助于在显存较小的设备上训练大模型。  \n- **fp16**：是否启用16位（混合精度）训练，相比默认的32位训练能加速训练并节省显存。  \n- **eval_steps**：每隔多少训练步数进行一次验证评估，以便观察模型在验证集上的表现。  \n- **learning_rate**：训练时使用的学习率，决定模型参数更新的幅度。  \n- **logging_dir**：存储训练过程中的日志信息的目录，可以用来追踪和分析训练过程。  \n- **logging_steps**：每隔多少步记录一次训练日志，帮助监控训练进度。  \n- **run_name**：这次训练运行的名称，方便在日志和管理界面中区分不同的实验。\n'

In [55]:
training_args = TrainingArguments(
    output_dir='./fine_tuned_models',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=1, # batch size per device during training
    gradient_accumulation_steps=8, # number of updates steps to accumulate before performing a backward/update pass
    fp16=True,                     # GPUWhether to use 16-bit (mixed) precision training instead of 32-bit training.
    eval_steps=10,                 # Number of update steps between two evaluations.
    learning_rate=5e-5,            # learning rate used for training
    logging_dir='./logs',          # directory for storing logs
    logging_steps=10,
    run_name='deepseek-r1-sft-distill',
)

# Trainer

In [53]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_dataset,
    eval_dataset=tokenized_eval_dataset,
)

ValueError: fp16 mixed precision requires a GPU (not 'mps').

# Train

In [ ]:
trainer.train()
#model.save_pretrained(model_name)
#tokenizer.save_pretrained(model_name)

# Save trained model

## lora model

In [ ]:
save_path = './lora_models'
# model.save_pretrained(save_path)
# tokenizer.save_pretrained(save_path)

## base model

In [ ]:
final_save_path = './final_models'

base_model = AutoModelForCausalLM.from_pretrained(model_name)
model = PeftModel.from_pretrained(base_model, save_path)
model = model.merge_and_unload()

model.save_pretrained(final_save_path)
tokenizer.save_pretrained(final_save_path)


# Inference

## load model

In [ ]:
model = AutoModelForCausalLM.from_pretrained(final_save_path)
tokenizer = AutoTokenizer.from_pretrained(final_save_path)


## inference pipeline

In [ ]:
pipe = pipeline('text-generation', model=model, tokenizer=tokenizer)


In [ ]:
prompt = "What is the meaning of life?"

In [ ]:
generated_text = pipe(prompt, max_length=50, num_return_sequences=1, do_sample=True, temperature=0.8)

In [ ]:
print(generated_text[0]['generated_text'])   